In [163]:
import pandas as pd
import zipfile
from pathlib import Path
import json
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
root_dir = Path(r'C:\Users\97254\PycharmProjects\nlp_project_results\dev_dataset\train_on_80_percent_of_group_names')

In [49]:
EVAL_RESULTS_ZIP = root_dir /'cyber_attrs_eval_by_model_df.zip'
EVAL_GT_ZIP = root_dir /'cyber_attrs_eval_df.zip'

METRICS_JSON = root_dir /'metrics.json'

In [50]:
def extract_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(zip_path.parent)
    return zip_path.with_suffix('.csv')

In [113]:
EVAL_RESULTS_CSV = extract_zip(EVAL_RESULTS_ZIP)
EVAL_GT_CSV = extract_zip(EVAL_GT_ZIP)

In [114]:
EVAL_RESULTS_CSV

WindowsPath('C:/Users/97254/PycharmProjects/nlp_project_results/dev_dataset/train_on_80_percent_of_group_names/cyber_attrs_eval_by_model_df.csv')

In [115]:
EVAL_GT_CSV

WindowsPath('C:/Users/97254/PycharmProjects/nlp_project_results/dev_dataset/train_on_80_percent_of_group_names/cyber_attrs_eval_df.csv')

In [239]:
original_tstr_df = pd.read_csv(EVAL_RESULTS_CSV)[['Unnamed: 0', 'git_repo', 'year', 'filename', 'group_name']]

print(tstr_df.shape)
tstr_df.head()

(3386, 5)


,Unnamed: 0,git_repo,year,filename,group_name
0,0,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[]
1,1,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[]
2,2,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[]
3,3,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[]
4,4,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[]


In [240]:
original_tstgt_df = pd.read_csv(EVAL_GT_CSV)[['Unnamed: 0', 'git_repo', 'year', 'filename', 'group_name']]
print(tstgt_df.shape)
tstgt_df.head()

(3386, 5)


,Unnamed: 0,git_repo,year,filename,group_name
0,0,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[Hidden Cobra]
1,1,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[Hidden Cobra]
2,2,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[Hidden Cobra]
3,3,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[Hidden Cobra]
4,4,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[Hidden Cobra]


In [241]:
tstgt_df["group_name"] = original_tstgt_df["group_name"].apply(eval)
tstr_df["group_name"] = original_tstr_df["group_name"].apply(eval)

In [191]:
series_of_list = list(tstgt_df["group_name"])
one_list = [x for _list in series_of_list for x in _list]
all_group_names = set(one_list)
print("num of group_names: ", len(all_group_names))

num of group_names:  116


In [349]:
series_of_list_r = list(tstr_df["group_name"])
one_list_r = [x for _list in series_of_list_r for x in _list]
all_group_names_r = set(one_list_r)
new_names = all_group_names_r-all_group_names
print("num of new_names: ", len(new_names))

num of new_names:  22


In [23]:
labels = ["group_name", "software_name", "technique_name", "tactic"]
print(f"Total lines {len(tstr_df)}")
for label in labels:
    print(f"lines with {label}: {len(tstr_df[tstr_df[label] != str([])])}")

Total lines 3386
lines with group_name: 2832
lines with software_name: 103
lines with technique_name: 34
lines with tactic: 46


In [358]:
def check_if_label_exists(label, line):
    if label in line:
        return True
    return False

In [406]:
def create_stats_df(gt_df, results_df, labels):
    verbose_gt_df = gt_df.copy()
    verbose_r_df = results_df.copy()
    
    value_counter = dict()
    verbose_lines = dict()

    for group in labels:
        verbose_gt_df[group] = original_tstgt_df.group_name.str.contains(group)
        verbose_r_df[group] = original_tstr_df.group_name.str.contains(group)

    for label_value in labels:
        t = (verbose_gt_df[label_value]==True)
        tp = (verbose_r_df[label_value]==True) & (verbose_gt_df[label_value]==True)
        fp = (verbose_r_df[label_value]==True) & (verbose_gt_df[label_value]==False)
        fn = (verbose_r_df[label_value]==False) & (verbose_gt_df[label_value]==True)
        tp=tp[tp==True]
        t=t[t==True]
        fp=fp[fp==True]
        fn=fn[fn==True]
        counts_ = [len(t),len(tp), len(fp), len(fn)]
        value_counter[label_value] = counts_
        verbose_lines[label_value] = {'tp': set(tp.index), 'tn': set(tn.index),'fp': set(fp.index),'fn': set(fn.index)}

    value_counter_df = pd.DataFrame.from_dict(value_counter).transpose()
    value_counter_df.columns =['t', 'tp', 'fp','fn']
    return value_counter_df, verbose_lines

In [407]:
value_counter_df,  verbose_lines= create_stats_df(tstgt_df, tstr_df,original_tstgt_df,original_tstr_df, all_group_names)

In [408]:
value_counter_df

,t,tp,fp,fn
OCEANLOTUS GROUP,1,0,0,1
dynamite panda,1,0,0,1
ke3chang,1,0,0,1
APT30,208,208,0,0
Suckfly,22,22,0,0
...,...,...,...,...
turla,8,0,0,8
Lazarus Group,379,379,0,0
APT18,23,23,0,0
LAZARUS GROUP,4,0,0,4


In [410]:
value_counter_df_new,  verbose_lines_new= create_stats_df(tstgt_df, tstr_df, new_names)

In [411]:
value_counter_df_new

,t,tp,fp,fn
ESPIONAGE TARGETED,0,0,1,0
House,0,0,1,0
Birds,0,0,1,0
Contrary,0,0,1,0
Romance,0,0,1,0
,0,0,0,0
NAME,0,0,1,0
Shell,0,0,1,0
............................................................................................................................................... 72,0,0,1,0
APT38,0,0,1,0


In [412]:
for word, dict_ in verbose_lines_new.items():
    for key, val in dict_.items():
        if key == 'fp':
            print(word, val)

ESPIONAGE TARGETED {951}
House {1841}
Birds {8}
Contrary {378}
Romance {284}
 set()
NAME {1471}
Shell {43}
............................................................................................................................................... 72 {3058}
APT38 {3352}
DarkHotel group {774}
TG-3279 {1274}
ZJ {2096}
BACKSPACE {2051, 2052, 2467, 2060, 2064, 2096, 2449}
COMPONENT {1438}
RemoveResources {1132}
Propaganda {586}
RIPTIDE {2915, 1205, 1199}
the Storm {1074}
collective set()
Overlapping {742}
The Bear {1071}


In [416]:
tstgt_df.columns

Index(['Unnamed: 0', 'git_repo', 'year', 'filename', 'group_name'], dtype='object')

In [427]:
verbose_gt_df = tstgt_df.copy().drop( ['Unnamed: 0','git_repo', 'year', 'filename','group_name'],axis='columns')
verbose_r_df = tstr_df.copy().drop(['Unnamed: 0','git_repo', 'year', 'filename','group_name'],axis='columns')
for group in all_group_names:
    verbose_gt_df[group] = original_tstgt_df.group_name.str.contains(group)
    verbose_r_df[group] = original_tstr_df.group_name.str.contains(group)


In [428]:
(verbose_r_df==True) & (verbose_gt_df==True)

,OCEANLOTUS GROUP,dynamite panda,ke3chang,APT30,Suckfly,menupass,apt29,stone panda,PATCHWORK,Hidden Cobra,...,LOTUS BLOSSOM,comment crew,cleaver,energetic bear,COMMENT PANDA,turla,Lazarus Group,APT18,LAZARUS GROUP,taidoor
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3382,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3383,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3384,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [464]:
def calc_general_stats(gt_df,r_df,original_gt_df,original_r_df,labels):
    verbose_gt_df = gt_df.copy().drop( ['Unnamed: 0','git_repo', 'year', 'filename','group_name'],axis='columns')
    verbose_r_df = r_df.copy().drop(['Unnamed: 0','git_repo', 'year', 'filename','group_name'],axis='columns')
    for group in labels:
        verbose_gt_df[group] = original_gt_df.group_name.str.contains(group)
        verbose_r_df[group] = original_r_df.group_name.str.contains(group)
    general_tp = (verbose_r_df==True) & (verbose_gt_df==True)
    general_tp=general_tp.sum(axis=1).sum()
    general_tn =  (verbose_r_df.sum(axis=1)==0) & (verbose_gt_df.sum(axis=1)==0)
    general_tn = general_tn.sum()
    general_fn =   (verbose_r_df==False) & (verbose_gt_df==True)
    general_fn=general_fn.sum(axis=1).sum()
    general_fp =   (verbose_r_df==True) & (verbose_gt_df==False)
    general_fp=general_fp.sum().sum()
    stats = {"tp": general_tp, "tn": general_tn, "fn": general_fn, "fp": general_fp}
    return stats

In [465]:
 calc_general_stats(tstgt_df, tstr_df,original_tstgt_df,original_tstr_df,all_group_names)

{'tp': 2868, 'tn': 0, 'fn': 580, 'fp': 0}

In [466]:
 calc_general_stats(tstgt_df, tstr_df,original_tstgt_df,original_tstr_df,new_names)

{'tp': 0, 'tn': 3359, 'fn': 0, 'fp': 28}